# Imports

In [3]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

# Explore Database

In [4]:
# Connect to database
path_to_db = "../data/DB/olist.db"
connection = sqlite3.connect(path_to_db)

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

We will start by exploring the table related to customers : customers.

In [5]:
query = """
    SELECT c.customer_unique_id, c.customer_id, c.customer_zip_code_prefix, c.customer_city, c.customer_state
    FROM customers c
"""
#  ,g.geolocation_lat, g.geolocation_lng
# LEFT JOIN geoloc g ON c.customer_zip_code_prefix = g.geolocation_zip_code_prefix

In [6]:
# Display the results in a pandas dataframe
df_customers = pd.read_sql_query(query, connection)
df_customers.head()

,customer_unique_id,customer_id,customer_zip_code_prefix,customer_city,customer_state
0,861eff4711a542e4b93843c6dd7febb0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP
1,290c77bc529b7ac935b93aa66c333dc3,18955e83d337fd6b2def6b18a428ac77,9790,sao bernardo do campo,SP
2,060e732b5b29e8181a18229c7b0b2b5e,4e7b3e00288586ebd08712fdd0374a03,1151,sao paulo,SP
3,259dac757896d24d7702b9acbbff3f3c,b2b6027bc5c5109e529d4dc6358b12c3,8775,mogi das cruzes,SP
4,345ecd01c38d18a9036ed96c73b8d066,4f2d8ab171c80ec8364f7c12e35b23ad,13056,campinas,SP


In [7]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_unique_id        99441 non-null  object
 1   customer_id               99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


## How many customers ? Where are they living ?

In [8]:
print(f"Number of customers : {len(df_customers['customer_unique_id'].unique())}")
print(f"Customers are from {len(df_customers['customer_state'].unique())} different states and {len(df_customers['customer_city'].unique())} different cities")

# get the number of orders per customer
orders_by_customer = df_customers.groupby("customer_unique_id")["customer_id"].count().sort_values(ascending=False)
# get min and max number of orders
min_orders = orders_by_customer.min()
max_orders = orders_by_customer.max()
# get the average number of orders
average_orders = orders_by_customer.mean()

print(f"The number of orders can be from {min_orders} to {max_orders} with an average of {round(average_orders, 2)} orders per customer")


Number of customers : 96096
Customers are from 27 different states and 4119 different cities
The number of orders can be from 1 to 17 with an average of 1.03 orders per customer


In [9]:
# Group customer by state and cities then count the number of customers in each group
customers_by_state = df_customers.groupby("customer_state")["customer_unique_id"].count().sort_values(ascending=False)

## Orders related informations

In [10]:
query = """
    SELECT c.customer_unique_id, c.customer_state, o.order_id, o.order_status, o.order_purchase_timestamp, o.order_approved_at, o.order_delivered_carrier_date, o.order_delivered_customer_date, o.order_estimated_delivery_date
    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
"""

df_orders = pd.read_sql_query(query, connection)
df_orders.head()

,customer_unique_id,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,861eff4711a542e4b93843c6dd7febb0,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00
1,290c77bc529b7ac935b93aa66c333dc3,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00
2,060e732b5b29e8181a18229c7b0b2b5e,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00
3,259dac757896d24d7702b9acbbff3f3c,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00
4,345ecd01c38d18a9036ed96c73b8d066,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00


### How often do they order ?

In [11]:
# For the customers with more than one order, get the number of days between an order and the previous one for each customer. The first order should be considered as 0
df_orders["order_purchase_timestamp"] = pd.to_datetime(df_orders["order_purchase_timestamp"])
df_orders = df_orders.sort_values(by=["customer_unique_id", "order_purchase_timestamp"])
df_grouped = df_orders.groupby("customer_unique_id")

result = df_grouped.agg(
    last_order_date=("order_purchase_timestamp", "max"),
    total_orders=("order_id", "count"),
)

# Average days between orders
df_orders["days_between_orders"] = (
    df_grouped["order_purchase_timestamp"].diff().dt.days.fillna(0).fillna(0)
)
average_days_between_orders = df_orders.groupby("customer_unique_id")["days_between_orders"].mean().fillna(0)

# Add average to results
result["average_days_between_orders"] = average_days_between_orders

result = result.reset_index()

result.head()

,customer_unique_id,last_order_date,total_orders,average_days_between_orders
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,1,0.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,1,0.0
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,1,0.0
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,1,0.0
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,1,0.0


In [12]:
# split last order date into date and time
result["last_order_time"] = result["last_order_date"].dt.time
result["last_order_date"] = result["last_order_date"].dt.date

result.head()

,customer_unique_id,last_order_date,total_orders,average_days_between_orders,last_order_time
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10,1,0.0,10:56:27
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07,1,0.0,11:11:27
2,0000f46a3911fa3c0805444483337064,2017-03-10,1,0.0,21:05:03
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12,1,0.0,20:29:41
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14,1,0.0,19:45:42


In [13]:
result.describe()

,total_orders,average_days_between_orders
count,96096.000000,96096.000000
mean,1.034809,1.275598
std,0.214384,11.984673
min,1.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,0.000000
75%,1.000000,0.000000
max,17.000000,304.000000


Who are the most recent customers ?

In [14]:
df_orders.sort_values(by="order_purchase_timestamp", ascending=False).head(10)

,customer_unique_id,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,days_between_orders
85415,87ab9fec999db8bd5774917de3cdf01c,SP,10a045cdf6a5650c21e9cfeb60384c16,canceled,2018-10-17 17:30:18,None,None,None,2018-10-30 00:00:00,0.0
69139,262e1f1e26e92e86375f86840b4ffd63,PI,b059ee4de278302d550a3035c4cdb740,canceled,2018-10-16 20:16:02,None,None,None,2018-11-12 00:00:00,127.0
68312,af5454198a97379394cacf676e1e96cb,SP,a2ac6dad85cf8af5b0afb510a240fe8c,canceled,2018-10-03 18:55:29,None,None,None,2018-10-16 00:00:00,30.0
75902,634420a0ea42302205032ed44ac7fccc,RJ,616fa7d4871b87832197b2a137a115d2,canceled,2018-10-01 15:30:09,None,None,None,2018-10-23 00:00:00,46.0
75985,9bb92bebd4cb7511e1a02d5e50bc4655,SP,392ed9afd714e3c74767d0c4d3e3f477,canceled,2018-09-29 09:13:03,None,None,None,2018-10-15 00:00:00,0.0
75807,ba84da8c159659f116329563a0a981dd,RJ,869997fbe01f39d184956b5c6bccfdbe,canceled,2018-09-26 08:40:15,None,None,None,2018-10-25 00:00:00,22.0
33139,9c3af16efacb7aa06aa3bc674556c5d6,MG,5aac76cf7b07dd06fa4d50bf461d2f40,canceled,2018-09-25 11:59:18,None,None,None,2018-10-11 00:00:00,53.0
41350,08642cd329066fe11ec63293f714f2f8,MG,ed3efbd3a87bea76c2812c66a0b32219,canceled,2018-09-20 13:54:16,None,None,None,2018-10-17 00:00:00,60.0
1087,ef0103e9602d12594d19c2b666219bc1,MG,bd35b677fd239386e9861d11ae98ab56,canceled,2018-09-17 17:21:16,None,None,None,2018-10-01 00:00:00,5.0
70913,c1ee153508c6b785b491443a95ff364e,SC,ea844c92cf978ea23321fa7fe5871761,canceled,2018-09-13 09:56:12,None,None,None,2018-09-27 00:00:00,0.0


In [15]:
df_orders["order_status"].unique()

array(['delivered', 'canceled', 'shipped', 'unavailable', 'invoiced',
       'processing', 'created', 'approved'], dtype=object)

We'd like to keep all categories to keep track on clients that ordered abd are waiting for they order to be approved and to keep track on customers that canceled their order. What is "unavailable" ? Is this the information ? We can work on this information to check if it was delivered

In [16]:
df_orders.loc[df_orders["order_status"] == "unavailable"].info()

<class 'pandas.core.frame.DataFrame'>
Index: 609 entries, 83995 to 96595
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   customer_unique_id             609 non-null    object        
 1   customer_state                 609 non-null    object        
 2   order_id                       609 non-null    object        
 3   order_status                   609 non-null    object        
 4   order_purchase_timestamp       609 non-null    datetime64[ns]
 5   order_approved_at              609 non-null    object        
 6   order_delivered_carrier_date   0 non-null      object        
 7   order_delivered_customer_date  0 non-null      object        
 8   order_estimated_delivery_date  609 non-null    object        
 9   days_between_orders            609 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 52.3+ KB


It seems that the order nas never delivered, neither to the customer, neither to the carrier. It concerns 609 orders.

## Are these orders related to a review ?

In [56]:
reviews_query = """
    SELECT o.order_id, c.customer_unique_id, o.order_status, r.review_score, r.review_comment_message, r.review_creation_date
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    LEFT JOIN order_reviews r ON o.order_id = r.order_id
"""

order_reviews = pd.read_sql_query(reviews_query, connection)
order_reviews.head()

,order_id,customer_unique_id,order_status,review_score,review_comment_message,review_creation_date
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,delivered,4.0,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,delivered,4.0,Muito bom o produto.,2018-08-08 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,delivered,5.0,None,2018-08-18 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,delivered,5.0,O produto foi exatamente o que eu esperava e e...,2017-12-03 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,72632f0f9dd73dfee390c9b22eb56dd6,delivered,5.0,None,2018-02-17 00:00:00


In [18]:
unavailable_order_reviews = order_reviews.loc[order_reviews["order_status"] == "unavailable"]
unavailable_order_reviews["review_score"].value_counts()

review_score
1.0    463
2.0     43
5.0     36
3.0     35
4.0     20
Name: count, dtype: int64

In [19]:
unavailable_order_reviews.loc[unavailable_order_reviews["review_score"] == 4]

,order_id,customer_unique_id,order_status,review_score,review_comment_message
2329,86f0bcd9b426a2c1022ae317c7f27be6,930c4390af58f67334447c3a1cf2ba36,unavailable,4.0,None
8324,0bde826daa0bd10393e958cf5d69dd26,9930a089a21d7d6754347faf03001f6f,unavailable,4.0,io
17427,151e44f3dd461fe0838b6f59f884749f,76c97e01b5a2a58b8456c4db49bd988b,unavailable,4.0,None
26186,00bca4adac549020c1273714d04d0208,6968d01009b952ed987de42a239d7fa0,unavailable,4.0,None
33558,a2bcbea8668a0d93334cf7f7fafe921d,db44a2fcc338d9b3d22dda89950c77fa,unavailable,4.0,chegou tudo conforme combinado.
33711,e9e56ab7d8e993bda2988ed430524126,ee1f5271e9485b974a603453a9df1fac,unavailable,4.0,None
36355,deb556fbebd4343315d9ab970392674c,ac3dcee918e6c96eb2ab467158143697,unavailable,4.0,None
39280,6dd0cc64fed627388bf95943af884f70,b8b3c435a58aebd788a477bed8342910,unavailable,4.0,None
41897,6d9bd9b4caa7d961a6d31427ad0e557e,19046f948d6229d3152989f959827c99,unavailable,4.0,None
43314,1c3a814fa046dc2f684276b1ca26786e,7d373e92dd3086b4c37e9868fc8999c1,unavailable,4.0,None


Might decide to drop those 609 rows...

## Do customers leave reviews and rate the product ?

In [57]:
# Get the total of reviews given by a customer and the average review score
reviews_by_customer = order_reviews.groupby("customer_unique_id")["review_score"].agg(
    total_reviews="count", average_review_score="mean"
)
reviews_by_customer = reviews_by_customer.reset_index()
reviews_by_customer.head()

,customer_unique_id,total_reviews,average_review_score
0,0000366f3b9a7992bf8c76cfdf3221e2,1,5.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,4.0
2,0000f46a3911fa3c0805444483337064,1,3.0
3,0000f6ccb0745a6a4b88665a16c9f078,1,4.0
4,0004aac84e0df4da2b147fca70cf8255,1,5.0


In [58]:
# Get the review length
order_reviews["review_length"] = order_reviews["review_comment_message"].str.len()
order_reviews["review_length"] = order_reviews["review_length"].fillna(0)
order_reviews.describe()

,review_score,review_length
count,99224.000000,99992.000000
mean,4.086421,28.127910
std,1.347579,48.246965
min,1.000000,0.000000
25%,4.000000,0.000000
50%,5.000000,0.000000
75%,5.000000,41.000000
max,5.000000,208.000000


In [59]:
order_reviews.loc[(order_reviews["review_length"] < 6)].sort_values(by="review_length", ascending=False)

,order_id,customer_unique_id,order_status,review_score,review_comment_message,review_creation_date,review_length
58892,0f1a032292563a581f27eac807f97a05,b5b2a7fd4d491c96d488db899e42e0e2,delivered,5.0,Otima,2018-07-31 00:00:00,5.0
22189,d5499e385aef94ea76d2ea5e6a73e60a,0d1b072f720c3e2778a1b93b937ec8ac,delivered,5.0,otimo,2017-03-04 00:00:00,5.0
54021,0eb7dc4e9168fd2e3bc791983492d99a,22f2ac2a52ef231ad6d14660da9054ee,delivered,5.0,Ótimo,2018-07-13 00:00:00,5.0
28262,c588d6fe9640b6198171ed8c30981057,c49003818eeffa06a089d621c92f05c0,delivered,5.0,otimo,2018-07-10 00:00:00,5.0
96735,eddc2675550fd30e8c067b3fdd08ad60,b903cf485dbbbca3620a415d018565f6,delivered,5.0,Otimo,2017-11-23 00:00:00,5.0
...,...,...,...,...,...,...,...
33769,b1a000260e2f3760e5ec18566db7d592,0f2bf926e0f3cdc9092f3388063eec2e,delivered,4.0,None,2017-06-30 00:00:00,0.0
33772,85b4db0acc8f67917cb9245d8547ecaa,fe40c8ed5d902c8ec1032044a223b285,shipped,1.0,None,2018-03-24 00:00:00,0.0
33773,10856b02e17c5d56a06c2ee75079d338,6c00d2b9180327258ff03b6acd57ded1,delivered,5.0,None,2018-01-11 00:00:00,0.0
33774,9645c5cc1c315607f2524b2189d3efe1,be96f24255f8b90d5401437eb2f56c83,delivered,5.0,None,2017-07-07 00:00:00,0.0


In [60]:
order_reviews["left_review"] = order_reviews.apply(lambda x: 1 if x["review_length"] > 1 else 0, axis=1)
order_reviews.drop(columns=["review_length", "review_comment_message"], inplace=True)
order_reviews.head()

,order_id,customer_unique_id,order_status,review_score,review_creation_date,left_review
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,delivered,4.0,2017-10-11 00:00:00,1
1,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,delivered,4.0,2018-08-08 00:00:00,1
2,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,delivered,5.0,2018-08-18 00:00:00,0
3,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,delivered,5.0,2017-12-03 00:00:00,1
4,ad21c59c0840e6cb83a9ceb5573f8159,72632f0f9dd73dfee390c9b22eb56dd6,delivered,5.0,2018-02-17 00:00:00,0


In [61]:
# add to the result dataframe
result = result.merge(reviews_by_customer, on="customer_unique_id", how="left")
result.head()

,customer_unique_id,last_order_date,total_orders,average_days_between_orders,last_order_time,total_reviews_x,average_review_score_x,total_reviews_y,average_review_score_y,total_reviews,average_review_score
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10,1,0.0,10:56:27,1,5.0,1,5.0,1,5.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07,1,0.0,11:11:27,1,4.0,1,4.0,1,4.0
2,0000f46a3911fa3c0805444483337064,2017-03-10,1,0.0,21:05:03,1,3.0,1,3.0,1,3.0
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12,1,0.0,20:29:41,1,4.0,1,4.0,1,4.0
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14,1,0.0,19:45:42,1,5.0,1,5.0,1,5.0


## What do they order and how much they pay?

In [62]:
products_query = """
    SELECT c.customer_unique_id, o.order_id, oi.price, oi.freight_value, t.product_category_name_english, p.product_description_lenght, p.product_photos_qty
    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    JOIN products p ON oi.product_id = p.product_id
    LEFT JOIN translation t ON p.product_category_name = t.product_category_name
"""

df_orders_detail = pd.read_sql_query(products_query, connection)
df_orders_detail.head()

,customer_unique_id,order_id,price,freight_value,product_category_name_english,product_description_lenght,product_photos_qty
0,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,124.99,21.88,office_furniture,1141.0,1.0
1,290c77bc529b7ac935b93aa66c333dc3,29150127e6685892b6eab3eec79f59c7,289.00,46.48,housewares,1002.0,3.0
2,060e732b5b29e8181a18229c7b0b2b5e,b2059ed67ce144a36e2aa97d2c9e9ad2,139.94,17.79,office_furniture,955.0,1.0
3,259dac757896d24d7702b9acbbff3f3c,951670f92359f4fe4a63112aa7306eba,149.94,23.36,office_furniture,1066.0,1.0
4,345ecd01c38d18a9036ed96c73b8d066,6b7d50bd145f6fc7f33cebabd7e49d0f,230.00,22.25,home_confort,407.0,1.0


In [23]:
#

# Get the full informations

In [28]:
details_query = """
    SELECT
    c.customer_unique_id, c.customer_state, c.customer_city, c.customer_zip_code_prefix,
    o.order_id, o.order_status, o.order_purchase_timestamp, o.order_delivered_customer_date, o.order_estimated_delivery_date,
    pmt.payment_sequential, pmt.payment_type, pmt.payment_installments,
    r.review_score, r.review_comment_message,
    oi.price, oi.freight_value,
    p.product_description_lenght, p.product_photos_qty,
    t.product_category_name_english,
    s.seller_id, s.seller_city, s.seller_state, s.seller_zip_code_prefix
    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
    LEFT JOIN order_pymts pmt ON o.order_id = pmt.order_id
    LEFT JOIN order_reviews r ON o.order_id = r.order_id
    LEFT JOIN order_items oi ON o.order_id = oi.order_id
    LEFT JOIN sellers s ON oi.seller_id = s.seller_id
    LEFT JOIN products p ON oi.product_id = p.product_id
    LEFT JOIN translation t ON p.product_category_name = t.product_category_name
"""
df_details = pd.read_sql_query(details_query, connection)
df_details.head(10)

,customer_unique_id,customer_state,customer_city,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,...,review_comment_message,price,freight_value,product_description_lenght,product_photos_qty,product_category_name_english,seller_id,seller_city,seller_state,seller_zip_code_prefix
0,861eff4711a542e4b93843c6dd7febb0,SP,franca,14409,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-25 10:35:35,2017-06-05 00:00:00,1.0,...,None,124.99,21.88,1141.0,1.0,office_furniture,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,SP,8577.0
1,290c77bc529b7ac935b93aa66c333dc3,SP,sao bernardo do campo,9790,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-29 12:41:19,2018-02-06 00:00:00,1.0,...,None,289.00,46.48,1002.0,3.0,housewares,b8bc237ba3788b23da09c0f1f3a3288c,itajai,SC,88303.0
2,060e732b5b29e8181a18229c7b0b2b5e,SP,sao paulo,1151,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-06-14 17:58:51,2018-06-13 00:00:00,1.0,...,None,139.94,17.79,955.0,1.0,office_furniture,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,SP,8577.0
3,259dac757896d24d7702b9acbbff3f3c,SP,mogi das cruzes,8775,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-28 16:04:25,2018-04-10 00:00:00,1.0,...,None,149.94,23.36,1066.0,1.0,office_furniture,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,SP,8577.0
4,345ecd01c38d18a9036ed96c73b8d066,SP,campinas,13056,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-08-09 20:55:48,2018-08-15 00:00:00,1.0,...,O baratheon è esxelente Amo adoro o baratheon,230.00,22.25,407.0,1.0,home_confort,4a3ca9315b744ce9f8e9374361493884,ibitinga,SP,14940.0
5,4c93744516667ad3b8f1fb645a3116a4,SC,jaragua do sul,89254,5741ea1f91b5fbab2bd2dc653a5b5099,delivered,2017-09-14 18:14:31,2017-09-28 17:32:43,2017-10-04 00:00:00,1.0,...,Loja responsável,259.90,22.31,282.0,2.0,sports_leisure,8f2ce03f928b567e3d56181ae20ae952,pirituba,SP,5141.0
6,addec96d2e059c80c30fe6871d30d177,SP,sao paulo,4534,36e694cf4cbc2a4803200c35e84abdc4,delivered,2018-02-19 14:38:35,2018-02-20 16:25:51,2018-03-05 00:00:00,1.0,...,None,14.99,7.78,322.0,3.0,computers_accessories,9f505651f4a6abe901a56cdc21508025,sao paulo,SP,4102.0
7,57b2a98a409812fe9618067b6b8ebe4f,MG,timoteo,35182,1093c8304c7a003280dd34598194913d,delivered,2017-11-16 19:29:02,2017-11-27 12:44:36,2017-12-08 00:00:00,1.0,...,None,19.90,16.11,NaN,NaN,None,2a7dc43cecabf23403078e2188437d1d,sao paulo,SP,4142.0
8,1175e95fb47ddff9de6b2b06188f7e0d,PR,curitiba,81560,1ebeea841c590e86a14a0d7a48e7d062,delivered,2018-01-18 12:35:44,2018-01-26 15:17:57,2018-02-20 00:00:00,1.0,...,chegou antes do prometido,25.00,14.10,739.0,5.0,toys,95e03ca3d4146e4011985981aeb959b9,rio de janeiro,RJ,21210.0
9,9afe194fb833f79e300e37e580171f22,MG,belo horizonte,30575,7433cbcc783205509d66a5260da5b574,delivered,2018-01-08 11:22:34,2018-01-13 14:51:55,2018-02-05 00:00:00,1.0,...,"ótimo, entregou antes da data prevista.",99.90,22.57,272.0,1.0,furniture_decor,0241d4d5d36f10f80c644447315af0bd,curitiba,PR,80330.0
